<a href="https://colab.research.google.com/github/mralamdari/CV-Object-Detection-Projects/blob/main/Flower_Recognition_Challenges.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**Flowers Recognition**
It is a dataset from kaggle and I will implement some of the best notebooks and compare them to learn the best pattern to solve this problem.
you can access the dataset or the notebooks in [here](https://www.kaggle.com/datasets/alxmamaev/flowers-recognition)

In [ ]:
import os
import cv2
import tqdm
import numpy as np
import pandas as pd
import seaborn as sns
from zipfile import ZipFile
from matplotlib import style

import matplotlib.pyplot as plt
style.use('fivethirtyeight')
sns.set(style='whitegrid', color_codes=True)

import warnings
warnings.filterwarnings('always')
warnings.filterwarnings('ignore')

from PIL import Image
import tensorflow as tf
from sklearn import preprocessing, model_selection, metrics, decomposition

###Data

In [ ]:
os.environ['KAGGLE_CONFIG_DIR'] = '/content/drive/MyDrive/'
!kaggle datasets download -d alxmamaev/flowers-recognition
!unzip \*.zip && rm *.zip

####1.Train/Test Split data first way
A simple code to only open folders and save them one by one in the data and label folder

In [ ]:
path_folder = '/content/flowers'

size = 224
data = []
label = []
data_names = []
id = 0
for folder in tqdm.tqdm(os.listdir(path_folder)):
  for files in tqdm.tqdm(os.listdir(os.path.join(path_folder, folder))):
    if files.endswith('jpg'):
      label.append(folder)
      img_path = os.path.join(path_folder, folder, files)
      data_names.append((id, img_path))
      img = cv2.imread(img_path, cv2.IMREAD_COLOR)
      im = cv2.resize(img, (size, size))
      data.append(im)
      id += 1
    else:
      continue

In [ ]:
categories = set(label)
categories, len(label)

In [ ]:
np.array(data).shape

(4317, 224, 224, 3)

In [ ]:
data = np.array(data)
label = np.array(label)

In [ ]:
x = data/255
encoder = preprocessing.LabelEncoder()
y = encoder.fit_transform(label)
y = tf.keras.utils.to_categorical(y)

Creating test, train data from x, will consume all them memory we have in colab,
so instead we can put **data_names** in to train/test splits.


Since vgg16 model, returns the exact names of the categories and not it's encoded form, we will use label as y



In [ ]:
x_train, x_test, y_train, y_test = model_selection.train_test_split(x, y, random_state=32, test_size=0.2)

# x_train, x_test, y_train, y_test = model_selection.train_test_split(data_names, y, random_state=32, test_size=0.2)

# x_train, x_test, y_train, y_test = model_selection.train_test_split(data_names, label, random_state=32, test_size=0.2)

In [ ]:
len(x_train), len(x_test)

(3453, 864)

In [ ]:
x_train[:20]

[(270, '/content/flowers/tulip/9446982168_06c4d71da3_n.jpg'),
 (3088, '/content/flowers/daisy/4837182901_69a6cc782b_n.jpg'),
 (3784, '/content/flowers/sunflower/5043409856_395300dbe5_m.jpg'),
 (238, '/content/flowers/tulip/17730239562_9aa0fc0738_n.jpg'),
 (1325, '/content/flowers/dandelion/19067907051_16d530c7d2.jpg'),
 (3828, '/content/flowers/sunflower/2950505226_529e013bf7_m.jpg'),
 (3313, '/content/flowers/daisy/2901376034_cdb4bac26b_m.jpg'),
 (911, '/content/flowers/tulip/6994351792_343e18cbf6_n.jpg'),
 (4228, '/content/flowers/sunflower/44079668_34dfee3da1_n.jpg'),
 (8, '/content/flowers/tulip/8454707381_453b4862eb_m.jpg'),
 (3030, '/content/flowers/daisy/5109508979_68e3530791_m.jpg'),
 (593, '/content/flowers/tulip/8586205168_8294e67195_n.jpg'),
 (3547, '/content/flowers/daisy/367020749_3c9a652d75.jpg'),
 (464, '/content/flowers/tulip/16986144192_55e0e6c152.jpg'),
 (1564, '/content/flowers/dandelion/6044710875_0459796d1b_m.jpg'),
 (2783, '/content/flowers/rose/18990187093_09f2bf

####Visualize data

In [ ]:
fig, ax = plt.subplots(4, 4)
fig.set_size_inches(20, 20)
for i in range(4):
  for j in range(4):
    rnd = np.random.randint(0, len(x), 1)[0]
    ax[i, j].imshow(x[rnd])
    ax[i, j].set_title(f'Flower {label[rnd]}')
plt.tight_layout()

# 1.CNN Architectures : VGG, ResNet, Inception + TL
[Note Book](https://www.kaggle.com/code/shivamb/cnn-architectures-vgg-resnet-inception-tl)

##1.1 VGG16
####Visual Geometry Group from Oxford
VGG-16 is a convolutional neural network that is 16 layers deep. You can load a pretrained version of the network trained on more than a million images from the ImageNet database. The pretrained network can classify images into 1000 object categories, such as keyboard, mouse, pencil, and many animals

In [ ]:
n_1, n_2, n_3, n_4 = np.random.randint(0, len(x_train), 4)
test_imgs = np.array([x_train[n_1], x_train[n_2], x_train[n_3], x_train[n_4]])
test_imgs_y = np.array([y_train[n_1], y_train[n_2], y_train[n_3], y_train[n_4]])

In [ ]:
def image_loader(img_path, t_size=224):
  img = tf.keras.preprocessing.image.load_img(img_path, target_size=(t_size, t_size))
  img = tf.keras.preprocessing.image.img_to_array(img)
  img = np.expand_dims(img, axis=0)
  img = tf.keras.applications.vgg16.preprocess_input(img)
  return img

def predict_img(imgs, imgs_y, model):
  fig, ax = plt.subplots(1, 4, figsize=(30, 40))  
  for i, img_path in enumerate(imgs):
    img = image_loader(img_path[1])
    preds = tf.keras.applications.vgg16.decode_predictions(model.predict(img), top=10)[0]
    flower_pred = None
    for p in preds:
      if p[1] in categories:
        flower_pred = p
        break
    final_pred = flower_pred if flower_pred else preds[0]

    ax[i].imshow(Image.open(imgs[i][1]).resize((200, 200), Image.ANTIALIAS))
    color = 'green' if final_pred[1] == imgs_y[i] else 'red'
    ax[i].set_title(f"Pred: {final_pred[1]} with: %{100*final_pred[-1]:.2f} Confidence", color= color, fontweight='bold')
  plt.show()

###From Scratch VGG16

In [ ]:
class VGG16(tf.keras.models.Model):
  def __init__(self):
    super(VGG16, self).__init__()

    #1
    self.conv1 = tf.keras.layers.Conv2D(filters=64, kernel_size=3, padding='same', activation='relu', name='block1_conv1')
    self.conv2 = tf.keras.layers.Conv2D(filters=64, kernel_size=3, padding='same', activation='relu', name='block1_conv2')
    self.maxpool1 = tf.keras.layers.MaxPool2D(pool_size=(2, 2))
    #2
    self.conv3 = tf.keras.layers.Conv2D(filters=128, kernel_size=3, padding='same', activation='relu', name='block2_conv1')
    self.conv4 = tf.keras.layers.Conv2D(filters=128, kernel_size=3, padding='same', activation='relu', name='block2_conv2')
    self.maxpool2 = tf.keras.layers.MaxPool2D(pool_size=(2, 2))
    #3
    self.conv5 = tf.keras.layers.Conv2D(filters=256, kernel_size=3, padding='same', activation='relu', name='block3_conv1')
    self.conv6 = tf.keras.layers.Conv2D(filters=256, kernel_size=3, padding='same', activation='relu', name='block3_conv2')
    self.conv7 = tf.keras.layers.Conv2D(filters=256, kernel_size=3, padding='same', activation='relu', name='block3_conv3')
    self.maxpool3 = tf.keras.layers.MaxPool2D(pool_size=(2, 2))
    #4
    self.conv8 = tf.keras.layers.Conv2D(filters=512, kernel_size=3, padding='same', activation='relu', name='block4_conv1')
    self.conv9 = tf.keras.layers.Conv2D(filters=512, kernel_size=3, padding='same', activation='relu', name='block4_conv2')
    self.conv10 = tf.keras.layers.Conv2D(filters=512, kernel_size=3, padding='same', activation='relu', name='block4_conv3')
    self.maxpool4 = tf.keras.layers.MaxPool2D(pool_size=(2, 2))
    #5
    self.conv11 = tf.keras.layers.Conv2D(filters=512, kernel_size=3, padding='same', activation='relu', name='block5_conv1')
    self.conv12 = tf.keras.layers.Conv2D(filters=512, kernel_size=3, padding='same', activation='relu', name='block5_conv2')
    self.conv13 = tf.keras.layers.Conv2D(filters=512, kernel_size=3, padding='same', activation='relu', name='block5_conv3')
    self.maxpool5 = tf.keras.layers.MaxPool2D(pool_size=(2, 2))
    #6
    self.flat = tf.keras.layers.Flatten()
    self.dense1 = tf.keras.layers.Dense(4096, activation='relu', name='fc1')
    self.droput1 = tf.keras.layers.Dropout(rate=0.5, name='Dropout1')
    self.dense2 = tf.keras.layers.Dense(4096, activation='relu', name='fc2')
    self.droput2 = tf.keras.layers.Dropout(rate=0.5, name='Dropout2')
    self.dense3 = tf.keras.layers.Dense(1000, activation='softmax', name='fc2')


  def call(self, inputs):
    # inputs = tf.keras.Input(shape=(224, 224, 1))
    x = self.conv1(inputs)
    x = self.conv2(x)
    x = self.maxpool1(x)

    x = self.conv3(x)
    x = self.conv4(x)
    x = self.maxpool2(x)

    x = self.conv5(x)
    x = self.conv6(x)
    x = self.conv7(x)
    x = self.maxpool3(x)

    x = self.conv8(x)
    x = self.conv9(x)
    x = self.conv10(x)
    x = self.maxpool4(x)

    x = self.conv11(x)
    x = self.conv12(x)
    x = self.conv13(x)
    x = self.maxpool5(x)
    
    x = self.flat(x)

    x = self.dense1(x)
    x = self.droput1(x)

    x = self.dense2(x)
    x = self.droput2(x)

    x = self.dense3(x)

    return x

In [ ]:
!wget https://github.com/fchollet/deep-learning-models/releases/download/v0.1/vgg16_weights_tf_dim_ordering_tf_kernels.h5

In [ ]:
vgg16_model = VGG16()
vgg16_model.build(input_shape=(1, 224, 224, 3))
vgg16_model.summary()

Model: "vgg16_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 block1_conv1 (Conv2D)       multiple                  1792      
                                                                 
 block1_conv2 (Conv2D)       multiple                  36928     
                                                                 
 max_pooling2d_10 (MaxPoolin  multiple                 0         
 g2D)                                                            
                                                                 
 block2_conv1 (Conv2D)       multiple                  73856     
                                                                 
 block2_conv2 (Conv2D)       multiple                  147584    
                                                                 
 max_pooling2d_11 (MaxPoolin  multiple                 0         
 g2D)                                                      

In [ ]:
vgg16_weights = 'vgg16_weights_tf_dim_ordering_tf_kernels.h5'
vgg16_model.load_weights('vgg16_weights_tf_dim_ordering_tf_kernels.h5')

In [ ]:
predict_img(test_imgs, test_imgs_y, vgg16_model)

###Pretrained VGG16

In [ ]:
def image_loader_pretrained(the_model, img_path, t_size=224):
  img = tf.keras.preprocessing.image.load_img(img_path, target_size=(t_size, t_size))
  img = tf.keras.preprocessing.image.img_to_array(img)
  img = np.expand_dims(img, axis=0)
  img = the_model.preprocess_input(img)
  return img

def predict_img_pretrained(imgs, imgs_y, model, the_model):
  fig, ax = plt.subplots(1, 4, figsize=(30, 40))  
  for i, img_path in enumerate(imgs):
    img = image_loader_pretrained(the_model, img_path[1])
    preds = the_model.decode_predictions(model.predict(img), top=10)[0]
    flower_pred = None
    for p in preds:
      if p[1] in categories:
        flower_pred = p
        break
    final_pred = flower_pred if flower_pred else preds[0]

    ax[i].imshow(Image.open(imgs[i][1]).resize((200, 200), Image.ANTIALIAS))
    color = 'green' if final_pred[1] == imgs_y[i] else 'red'
    ax[i].set_title(f"Pred: {final_pred[1]} with: %{100*final_pred[-1]:.2f} Confidence", color= color, fontweight='bold')
  plt.show()

In [ ]:
vgg16_model_pretrained = tf.keras.applications.VGG16(weights=vgg16_weights)
predict_img_pretrained(test_imgs, test_imgs_y, vgg16_model_pretrained, tf.keras.applications.vgg16)

##1.2 VGG19
####It is the VGG16 model with a small difference, there is one extra convolutional layer in each of the [3, 4, 5] blocks


###From Scratch VGG19

In [ ]:
class VGG19(tf.keras.models.Model):
  def __init__(self):
    super(VGG19, self).__init__()

    #1
    self.conv1 = tf.keras.layers.Conv2D(filters=64, kernel_size=3, padding='same', activation='relu', name='block1_conv1')
    self.conv2 = tf.keras.layers.Conv2D(filters=64, kernel_size=3, padding='same', activation='relu', name='block1_conv2')
    self.maxpool1 = tf.keras.layers.MaxPool2D(pool_size=(2, 2))
    #2
    self.conv3 = tf.keras.layers.Conv2D(filters=128, kernel_size=3, padding='same', activation='relu', name='block2_conv1')
    self.conv4 = tf.keras.layers.Conv2D(filters=128, kernel_size=3, padding='same', activation='relu', name='block2_conv2')
    self.maxpool2 = tf.keras.layers.MaxPool2D(pool_size=(2, 2))
    #3
    self.conv5 = tf.keras.layers.Conv2D(filters=256, kernel_size=3, padding='same', activation='relu', name='block3_conv1')
    self.conv6 = tf.keras.layers.Conv2D(filters=256, kernel_size=3, padding='same', activation='relu', name='block3_conv2')
    self.conv7 = tf.keras.layers.Conv2D(filters=256, kernel_size=3, padding='same', activation='relu', name='block3_conv3')
    self.conv8 = tf.keras.layers.Conv2D(filters=256, kernel_size=3, padding='same', activation='relu', name='block3_conv4')
    self.maxpool3 = tf.keras.layers.MaxPool2D(pool_size=(2, 2))
    #4
    self.conv9 = tf.keras.layers.Conv2D(filters=512, kernel_size=3, padding='same', activation='relu', name='block4_conv1')
    self.conv10 = tf.keras.layers.Conv2D(filters=512, kernel_size=3, padding='same', activation='relu', name='block4_conv2')
    self.conv11 = tf.keras.layers.Conv2D(filters=512, kernel_size=3, padding='same', activation='relu', name='block4_conv3')
    self.conv12 = tf.keras.layers.Conv2D(filters=512, kernel_size=3, padding='same', activation='relu', name='block4_conv4')
    self.maxpool4 = tf.keras.layers.MaxPool2D(pool_size=(2, 2))
    #5
    self.conv13 = tf.keras.layers.Conv2D(filters=512, kernel_size=3, padding='same', activation='relu', name='block5_conv1')
    self.conv14 = tf.keras.layers.Conv2D(filters=512, kernel_size=3, padding='same', activation='relu', name='block5_conv2')
    self.conv15 = tf.keras.layers.Conv2D(filters=512, kernel_size=3, padding='same', activation='relu', name='block5_conv3')
    self.conv16 = tf.keras.layers.Conv2D(filters=512, kernel_size=3, padding='same', activation='relu', name='block5_conv4')

    self.maxpool5 = tf.keras.layers.MaxPool2D(pool_size=(2, 2))
    #6
    self.flat = tf.keras.layers.Flatten()
    self.dense1 = tf.keras.layers.Dense(4096, activation='relu', name='fc1')
    self.droput1 = tf.keras.layers.Dropout(rate=0.5, name='Dropout1')
    self.dense2 = tf.keras.layers.Dense(4096, activation='relu', name='fc2')
    self.droput2 = tf.keras.layers.Dropout(rate=0.5, name='Dropout2')
    self.dense3 = tf.keras.layers.Dense(1000, activation='softmax', name='fc2')


  def call(self, inputs):
    # inputs = tf.keras.Input(shape=(224, 224, 1))
    x = self.conv1(inputs)
    x = self.conv2(x)
    x = self.maxpool1(x)

    x = self.conv3(x)
    x = self.conv4(x)
    x = self.maxpool2(x)

    x = self.conv5(x)
    x = self.conv6(x)
    x = self.conv7(x)
    x = self.conv8(x)
    x = self.maxpool3(x)

    x = self.conv9(x)
    x = self.conv10(x)
    x = self.conv11(x)
    x = self.conv12(x)
    x = self.maxpool4(x)

    x = self.conv13(x)
    x = self.conv14(x)
    x = self.conv15(x)
    x = self.conv16(x)
    x = self.maxpool5(x)
    
    x = self.flat(x)

    x = self.dense1(x)
    x = self.droput1(x)

    x = self.dense2(x)
    x = self.droput2(x)

    x = self.dense3(x)

    return x

In [ ]:
!wget https://github.com/fchollet/deep-learning-models/releases/download/v0.1/vgg19_weights_tf_dim_ordering_tf_kernels.h5

In [ ]:
vgg19_model = VGG19()
vgg19_model.build(input_shape=(1, 224, 224, 3))
vgg19_model.summary()

Model: "vgg19_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 block1_conv1 (Conv2D)       multiple                  1792      
                                                                 
 block1_conv2 (Conv2D)       multiple                  36928     
                                                                 
 max_pooling2d_25 (MaxPoolin  multiple                 0         
 g2D)                                                            
                                                                 
 block2_conv1 (Conv2D)       multiple                  73856     
                                                                 
 block2_conv2 (Conv2D)       multiple                  147584    
                                                                 
 max_pooling2d_26 (MaxPoolin  multiple                 0         
 g2D)                                                      

In [ ]:
vgg19_weights = 'vgg19_weights_tf_dim_ordering_tf_kernels.h5'
vgg19_model.load_weights(vgg19_weights)

In [ ]:
predict_img(test_imgs, test_imgs_y, vgg19_model)

###Pretrained VGG19

In [ ]:
vgg19_weights = 'vgg19_weights_tf_dim_ordering_tf_kernels.h5'
vgg19_model_pretrained = tf.keras.applications.VGG19(weights=vgg19_weights)
predict_img_pretrained(test_imgs, test_imgs_y, vgg19_model_pretrained, tf.keras.applications.vgg19)

##1.3 InceptionNets (GoogleNet)

## 1.4ResNet

##1.5 XceptionNets

# 2.Flower Recognition CNN Keras
[Note Book](https://www.kaggle.com/code/rajmehra03/flower-recognition-cnn-keras)

In [ ]:
datagen = tf.keras.preprocessing.image.ImageDataGenerator(
        featurewise_center=False,  # set input mean to 0 over the dataset
        samplewise_center=False,  # set each sample mean to 0
        featurewise_std_normalization=False,  # divide inputs by std of the dataset
        samplewise_std_normalization=False,  # divide each input by its std
        zca_whitening=False,  # apply ZCA whitening
        rotation_range=10,  # randomly rotate images in the range (degrees, 0 to 180)
        zoom_range = 0.1, # Randomly zoom image 
        width_shift_range=0.2,  # randomly shift images horizontally (fraction of total width)
        height_shift_range=0.2,  # randomly shift images vertically (fraction of total height)
        horizontal_flip=True,  # randomly flip images
        vertical_flip=False)  # randomly flip images

datagen.fit(x_train)

In [ ]:
model = tf.keras.models.Sequential()
model.add(tf.keras.layers.Conv2D(filters=32, kernel_size=5, padding='same',activation='relu', input_shape=(224, 224, 3)))
model.add(tf.keras.layers.MaxPooling2D(pool_size=2))

model.add(tf.keras.layers.Conv2D(filters=64, kernel_size=5, padding='same',activation='relu'))
model.add(tf.keras.layers.MaxPooling2D(pool_size=2))

model.add(tf.keras.layers.Conv2D(filters=128, kernel_size=5, padding='same',activation='relu'))
model.add(tf.keras.layers.MaxPooling2D(pool_size=2))

model.add(tf.keras.layers.Conv2D(filters=128, kernel_size=5, padding='same',activation='relu'))
model.add(tf.keras.layers.MaxPooling2D(pool_size=2))

model.add(tf.keras.layers.Flatten())
model.add(tf.keras.layers.Dense(512, activation='relu'))
model.add(tf.keras.layers.Dense(5, activation='softmax'))

In [ ]:
lr_callback =  tf.keras.callbacks.ReduceLROnPlateau(monitor='val_acc', pationce=3, verbose=1, factor=0.1)

In [ ]:
model.compile(optimizer = tf.keras.optimizers.Adam(lr=0.001),
              loss = 'categorical_crossentropy',
              metrics=['accuracy'])

In [ ]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 224, 224, 32)      2432      
                                                                 
 max_pooling2d (MaxPooling2D  (None, 112, 112, 32)     0         
 )                                                               
                                                                 
 conv2d_1 (Conv2D)           (None, 112, 112, 64)      51264     
                                                                 
 max_pooling2d_1 (MaxPooling  (None, 56, 56, 64)       0         
 2D)                                                             
                                                                 
 conv2d_2 (Conv2D)           (None, 56, 56, 128)       204928    
                                                                 
 max_pooling2d_2 (MaxPooling  (None, 28, 28, 128)      0

In [ ]:
batch_size = 32
history =model.fit(datagen.flow(x_train, y_train, batch_size=batch_size),
                   epochs=1, validation_data=(x_test, y_test), verbose=1)

In [ ]:
def plot_history(hist, loss='True'):
  title = 'loss' if loss else 'accuracy'
  plt.plot(hist.history[f'{title}'])
  plt.plot(hist.history[f'val_{title}'])
  plt.title(f'Model {title}')
  plt.ylabel(f'{title}')
  plt.xlabel('Epochs')
  plt.legend(['train', 'test'])
  plt.show()

In [ ]:
plot_history(history)

In [ ]:
plot_history(history, False)

In [ ]:
# getting predictions on val set.
pred=model.predict(x_test)
pred_digits=np.argmax(pred,axis=1)
# now storing some properly as well as misclassified indexes'.
i=0
prop_class=[]
mis_class=[]

for i in range(len(y_test)):
    if(np.argmax(y_test[i])==pred_digits[i]):
        prop_class.append(i)
    if(len(prop_class)==8):
        break

i=0
for i in range(len(y_test)):
    if(not np.argmax(y_test[i])==pred_digits[i]):
        mis_class.append(i)
    if(len(mis_class)==8):
        break

In [ ]:
count=0
fig,ax=plt.subplots(4,2)
fig.set_size_inches(15,15)
for i in range (4):
    for j in range (2):
        ax[i,j].imshow(x_test[prop_class[count]])
        ax[i,j].set_title("Predicted Flower : "+str(le.inverse_transform([pred_digits[prop_class[count]]]))+"\n"+"Actual Flower : "+str(le.inverse_transform(np.argmax([y_test[prop_class[count]]]))))
        plt.tight_layout()
        count+=1

In [ ]:
count=0
fig,ax=plt.subplots(4,2)
fig.set_size_inches(15,15)
for i in range (4):
    for j in range (2):
        ax[i,j].imshow(x_test[mis_class[count]])
        ax[i,j].set_title("Predicted Flower : "+str(le.inverse_transform([pred_digits[mis_class[count]]]))+"\n"+"Actual Flower : "+str(le.inverse_transform(np.argmax([y_test[mis_class[count]]]))))
        plt.tight_layout()
        count+=1